In [1]:
# Importing required libraries

import tensorflow as tf
import numpy as np

from tensorflow.keras import Model, layers
from sklearn.model_selection import train_test_split

2025-11-06 21:15:46.475014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762460146.496655     646 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762460146.502831     646 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762460146.522110     646 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762460146.522132     646 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762460146.522134     646 computation_placer.cc:177] computation placer alr

In [2]:
from tensorflow.keras.datasets import mnist

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to numpy float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize data between 0 and 1
#x_train, x_test = x_train/255, x_test/255

x_train = np.expand_dims(x_train, axis=-1)
x_test  = np.expand_dims(x_test, axis=-1)



# splitting Data
x_train, valid_data, y_train, valid_labels = train_test_split(
    x_train, y_train,
    test_size = 0.2,
    random_state = 42,
    shuffle=True
)

In [3]:
# Checking data
if x_train.shape[1] != 32:
    x_train = tf.pad(x_train, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)
    valid_data = tf.pad(valid_data, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)
    x_test = tf.pad(x_test, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)

print(x_train.shape)

I0000 00:00:1762460153.015033     646 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


(48000, 32, 32, 1)


In [4]:
# Parameters 
total_samples = len(x_train)
num_classes = 10

batch_size = 256
learning_rate = 0.001
epochs = 70
steps_per_epoch = int(np.ceil(total_samples / batch_size))

conv_layer1 = 6
conv_layer2 = 16

In [ ]:
# Data batching, splitting and shuffle

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(5000).repeat()


train_data = train_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [6]:
# Defining Model
class LeNet(tf.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        mu=0,
        sigma=0.01

        # Convolutional Layer 1
        self.w1 = tf.Variable(tf.random.normal([5, 5, 1, conv_layer1], mean=mu, stddev=sigma), name="weight1", trainable=True)
        self.b1 = tf.Variable(tf.zeros([conv_layer1]), name="bias1", trainable=True)

        # Convolutional Layer 2
        self.w2 = tf.Variable(tf.random.normal([5, 5, conv_layer1, conv_layer2], mean=mu, stddev=sigma), name="weight2", trainable=True)
        self.b2 = tf.Variable(tf.zeros([conv_layer2]), name="bias2", trainable=True)

        # Fully connected layer 1
        self.w3 = tf.Variable(tf.random.normal([5*5*conv_layer2, 120], mean=mu, stddev=sigma), name="weight3", trainable=True)
        self.b3 = tf.Variable(tf.zeros([120]), name="bias3", trainable=True)

        # Fully connected layer 2
        self.w4 = tf.Variable(tf.random.normal([120, 84], mean=mu, stddev=sigma), name="weight4", trainable=True)
        self.b4 = tf.Variable(tf.zeros([84]), name="bias4", trainable=True)

        # Output Layer
        self.w5 = tf.Variable(tf.random.normal([84, 10], mean=mu, stddev=sigma), name="weight5", trainable=True)
        self.b5 = tf.Variable(tf.zeros([10]), name="bias5", trainable=True)

        
    def conv2d(self, x, filter_W, bias_conv, stride=2, padding="VALID"):
        conv_layer = tf.nn.conv2d(
            x, filter_W,
            strides = [1, stride, stride, 1],
            padding = padding
        )
        conv_layer = tf.nn.bias_add(conv_layer, bias_conv)
        conv_layer = tf.nn.relu(conv_layer)
        return conv_layer

    def maxpool2d(self, x, k=2, s=2):
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding="SAME")
    
    def __call__(self, x, is_training=False):
        keep_prob = 0.7
        
        # Layer 1
        conv1 = self.conv2d(x, self.w1, self.b1, 1)
        conv1 = self.maxpool2d(conv1, 2, 2)

        # Layer 2
        conv2 = self.conv2d(conv1, self.w2, self.b2, 1)
        conv2 = self.maxpool2d(conv2, 2, 2)
        conv2 = tf.nn.dropout(conv2, keep_prob)

        # Flatten Layer 
        flatten = tf.reshape(conv2, [-1, self.w3.get_shape().as_list()[0]])

        # Fully connected layer 1
        fc1 = tf.nn.bias_add(tf.matmul(flatten, self.w3), self.b3)
        fc1 = tf.nn.relu(fc1)
        fc1 = tf.nn.dropout(fc1, keep_prob)

        # Fullly connected layer 2
        fc2 = tf.nn.bias_add(tf.matmul(fc1, self.w4), self.b4)
        fc2 = tf.nn.relu(fc2)
        fc1 = tf.nn.dropout(fc2, keep_prob)

        # Output layer
        out = tf.nn.bias_add(tf.matmul(fc2, self.w5), self.b5)

        if not is_training:
            return tf.nn.softmax(out)

        return out

In [7]:
optimizer = tf.optimizers.Adam(learning_rate)
model = LeNet()
checkpoint = tf.train.Checkpoint(model=model)
#checkpoint.save("./checkpoints/model")
manager = tf.train.CheckpointManager(checkpoint, "./checkpoints", max_to_keep=1)

In [8]:
def cross_entropy(pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=pred)
    return tf.reduce_mean(loss)

def accuracy(pred, y_true):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [9]:
def run_optimization(X, Y):
    with tf.GradientTape() as g:
        logits = model(X, is_training=True)
        loss = cross_entropy(logits, Y)

    gradient = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

In [10]:
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimization(batch_X, batch_Y)

    train_pred = model(batch_X, is_training=True)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    valid_pred = model(valid_data, is_training=True)
    valid_loss = cross_entropy(valid_pred, valid_labels)
    valid_acc = accuracy(valid_pred, valid_labels)
    manager.save()
    #if epoch == 40:
     #   learning_rate = 0.0005
     #   optimizer = tf.optimizers.Adam(learning_rate)
    #new_learning_rate = learning_rate * (0.95 ** epoch)
    #optimizer.learning_rate.assign(new_learning_rate)

    print(f"Epoch {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, validation_loss {valid_loss}, validation_accuracy {valid_acc}")

I0000 00:00:1762307695.055588   84382 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-11-05 02:55:08.692128: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, train_loss 0.5101274847984314, train_accuracy 0.84375, validation_loss 0.501129686832428, validation_accuracy 0.8422499895095825


2025-11-05 02:55:21.665332: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, train_loss 0.39101678133010864, train_accuracy 0.8671875, validation_loss 0.3663979172706604, validation_accuracy 0.8847500085830688
Epoch 3, train_loss 0.3229140639305115, train_accuracy 0.91015625, validation_loss 0.30673208832740784, validation_accuracy 0.909166693687439


2025-11-05 02:55:46.496268: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Epoch 16, train_loss 0.26372891664505005, train_accuracy 0.9375, validation_loss 0.21827135980129242, validation_accuracy 0.9356666803359985
Epoch 17, train_loss 0.13950099050998688, train_accuracy 0.95703125, validation_loss 0.19860273599624634, validation_accuracy 0.9424166679382324
Epoch 18, train_loss 0.14343607425689697, train_accuracy 0.953125, validation_loss 0.18535999953746796, validation_accuracy 0.9453333616256714
Epoch 19, train_loss 0.16334182024002075, train_accuracy 0.953125, validation_loss 0.1928251087665558, validation_accuracy 0.9441666603088379
Epoch 20, train_loss 0.22053247690200806, train_accuracy 0.93359375, validation_loss 0.21433258056640625, validation_accuracy 0.9389166831970215
Epoch 21, train_loss 0.09699929505586624, train_accuracy 0.97265625, validation_loss 0.19097131490707397, validation_accuracy 0.9483333230018616
Epoch 22, train_loss 0.16631640493869781, train_accuracy 0.94140625, validation_loss 0.18343643844127655, validation_accuracy 0.94808334112

2025-11-05 03:01:57.598610: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 32, train_loss 0.1616593301296234, train_accuracy 0.95703125, validation_loss 0.18028464913368225, validation_accuracy 0.9463333487510681
Epoch 33, train_loss 0.2102922648191452, train_accuracy 0.9375, validation_loss 0.17720744013786316, validation_accuracy 0.9462500214576721
Epoch 34, train_loss 0.16873976588249207, train_accuracy 0.95703125, validation_loss 0.18319739401340485, validation_accuracy 0.9483333230018616
Epoch 35, train_loss 0.19837510585784912, train_accuracy 0.94140625, validation_loss 0.1704270988702774, validation_accuracy 0.9488333463668823
Epoch 36, train_loss 0.19907930493354797, train_accuracy 0.94140625, validation_loss 0.17281949520111084, validation_accuracy 0.9482499957084656
Epoch 37, train_loss 0.16142235696315765, train_accuracy 0.94921875, validation_loss 0.18384386599063873, validation_accuracy 0.9480833411216736
Epoch 38, train_loss 0.14307866990566254, train_accuracy 0.9609375, validation_loss 0.1830686777830124, validation_accuracy 0.94608330726

2025-11-05 03:08:47.412295: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 64, train_loss 0.09359841793775558, train_accuracy 0.96484375, validation_loss 0.156622052192688, validation_accuracy 0.953249990940094
Epoch 65, train_loss 0.20784251391887665, train_accuracy 0.9375, validation_loss 0.1624641716480255, validation_accuracy 0.9524999856948853
Epoch 66, train_loss 0.23441541194915771, train_accuracy 0.921875, validation_loss 0.16498924791812897, validation_accuracy 0.9524999856948853
Epoch 67, train_loss 0.11671153455972672, train_accuracy 0.96484375, validation_loss 0.16839270293712616, validation_accuracy 0.9520000219345093
Epoch 68, train_loss 0.14976687729358673, train_accuracy 0.9453125, validation_loss 0.1907287985086441, validation_accuracy 0.9465000033378601
Epoch 69, train_loss 0.14875608682632446, train_accuracy 0.95703125, validation_loss 0.1680448055267334, validation_accuracy 0.9524166584014893
Epoch 70, train_loss 0.1387072503566742, train_accuracy 0.94921875, validation_loss 0.1528671532869339, validation_accuracy 0.9568333625793457


In [14]:
test_pred = model(x_test, is_training=True)
test_loss = cross_entropy(test_pred, y_test)
test_acc = accuracy(test_pred, y_test)

print(f"Test loss {test_loss} Test accuracy {test_acc}")

Test loss 0.1400926113128662 Test accuracy 0.9595999717712402


In [15]:
print(len(x_test))

10000


In [16]:
test_acc * len(x_test)

<tf.Tensor: shape=(), dtype=float32, numpy=9596.0>